In [2]:
import os
import pandas as pd
from rdkit import Chem
from functools import reduce

In [3]:
#za testove održivosti stanica (viability)

In [4]:
#putanje
Eseji = os.path.join(os.path.expanduser("~"), "Desktop", "Eseji")
csv_folder = r"D:\Krov 2\DESKTOP\Tox21\Eseji\CSV_viab" 
sdf_folder = r"D:\Krov 2\DESKTOP\Tox21\Eseji\SDF_viab" 

In [5]:
csv_files = [f for f in os.listdir(csv_folder) if f.endswith(".csv")]
sdf_files = [f for f in os.listdir(sdf_folder) if f.endswith(".sdf")]

In [6]:
def extract_sdf_data_with_smiles(path):
    supplier = Chem.SDMolSupplier(path)
    data = []
    for mol in supplier:
        if mol is None:
            continue
        try:
            props = mol.GetPropsAsDict()

            original_smiles = props.get("SMILES", None)

            generated_smiles = Chem.MolToSmiles(mol)

            props["SMILES_ORIGINAL"] = original_smiles
            props["SMILES_GENERATED"] = generated_smiles

            data.append(props)
        except:
            continue
    return pd.DataFrame(data)

In [7]:
csv_dfs = {
    os.path.splitext(f)[0]: pd.read_csv(os.path.join(csv_folder, f))
    for f in csv_files
}
sdf_dfs = {
    os.path.splitext(f)[0]: extract_sdf_data_with_smiles(os.path.join(sdf_folder, f))
    for f in sdf_files
}

[13:23:51] Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4
[13:23:51] ERROR: Could not sanitize molecule ending on line 58881
[13:23:51] ERROR: Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4
[13:23:51] ERROR: Problems encountered parsing Mol data, M  END missing around line 60078
[13:23:51] ERROR: moving to the beginning of the next molecule
[13:23:51] skipping block at line 108867: 'BEGIN ATOM'
[13:23:51] skipping block at line 108869: 'BEGIN BOND'
[13:23:51] Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4
[13:23:51] ERROR: Could not sanitize molecule ending on line 124705
[13:23:51] ERROR: Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4
[13:23:51] Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4
[13:23:51] ERROR: Could not sanitize molecule ending on line 137075
[13:23:51] ERROR: Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4
[13:23:51] ERROR: Problems encountered parsing Mol data, M  END missing around line 144305
[13:23:51] ERROR: moving to the beginning of the next mo

In [8]:
merged_dfs = {}
for key in sdf_dfs:
    if key in csv_dfs:
        sdf_df = sdf_dfs[key][["PREFERRED_NAME", "CASRN", "SMILES_ORIGINAL", "SMILES_GENERATED"]]
        csv_df = csv_dfs[key][["PREFERRED NAME", "CASRN", "HIT CALL"]]
        merged = pd.merge(
            sdf_df,
            csv_df,
            left_on=["PREFERRED_NAME", "CASRN"],
            right_on=["PREFERRED NAME", "CASRN"],
            how="inner"
        )
        merged = merged.drop(columns=["PREFERRED NAME"])
        merged = merged.rename(columns={"HIT CALL": f"{key}_HIT CALL"})
        merged_dfs[key] = merged

In [9]:
if merged_dfs:
    final_df = reduce(
    lambda left, right: pd.merge(left, right, on=["PREFERRED_NAME", "CASRN", "SMILES_GENERATED", "SMILES_ORIGINAL"], how="outer"),
    merged_dfs.values()
    )
    display(final_df)
else:
    print("Nema spojenih tablica! Provjeri poklapanje naziva i sadržaja CSV i SDF datoteka.")

,PREFERRED_NAME,CASRN,SMILES_ORIGINAL,SMILES_GENERATED,TOX21_AhR_LUC_Agonist_viability_HIT CALL,TOX21_AP1_BLA_Agonist_viability_HIT CALL,TOX21_ARE_BLA_agonist_viability_HIT CALL,TOX21_Aromatase_Inhibition_viability_HIT CALL,TOX21_AR_BLA_Antagonist_viability_HIT CALL,TOX21_AR_LUC_MDAKB2_Agonist_3uM_Nilutamide_viability_HIT CALL,...,TOX21_SBE_BLA_Antagonist_viability_HIT CALL,TOX21_SHH_3T3_GLI3_Agonist_viability_HIT CALL,TOX21_SHH_3T3_GLI3_Antagonist_viability_HIT CALL,TOX21_TRB_BLA_Agonist_Followup_viability_HIT CALL,TOX21_TRB_BLA_Antagonist_Followup_viability_HIT CALL,TOX21_TR_LUC_GH3_Antagonist_viability_HIT CALL,TOX21_TR_RXR_BLA_Agonist_Followup_viability_HIT CALL,TOX21_TR_RXR_BLA_Antagonist_Followup_viability_HIT CALL,TOX21_VDR_BLA_Agonist_viability_HIT CALL,TOX21_VDR_BLA_Antagonist_viability_HIT CALL
0,"((3,4-Dihydroxyphenyl)methylene)-Propanedinitrile",118409-57-7,OC1=C(O)C=C(C=C(C#N)C#N)C=C1,N#CC(C#N)=Cc1ccc(O)c(O)c1,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,...,Inactive,Inactive,Inactive,NaN,NaN,Inactive,NaN,NaN,Inactive,Inactive
1,(+)-2-Methylbutyl-4-methoxybenzyladine-4'-amin...,24140-30-5,CC[C@H](C)COC(=O)\C=C\C1=CC=C(C=C1)\N=C\C1=CC=...,CC[C@H](C)COC(=O)/C=C/c1ccc(/N=C/c2ccc(OC)cc2)cc1,Inactive,Active,Inactive,Active,Active,Inactive,...,Inactive,Inactive,Active,NaN,NaN,Inactive,NaN,NaN,Active,Active
2,(+)-Cedrol,77-53-2,[H][C@@]12CC[C@@H](C)[C@]11C[C@]([H])(C2(C)C)[...,C[C@@H]1CC[C@H]2C(C)(C)[C@H]3C[C@@]12CC[C@@]3(C)O,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,...,Inactive,Inactive,Inactive,NaN,NaN,Active,NaN,NaN,Inactive,Inactive
3,(+)-Cyclazocine,7313-87-3,C[C@@H]1[C@@H]2CC3=C(C=C(O)C=C3)[C@@]1(C)CCN2C...,C[C@@H]1[C@@H]2Cc3ccc(O)cc3[C@@]1(C)CCN2CC1CC1,Inactive,NaN,NaN,Inactive,Inactive,NaN,...,NaN,NaN,NaN,NaN,NaN,Inactive,NaN,NaN,NaN,NaN
4,(+)-Diltiazem,42399-41-7,COC1=CC=C(C=C1)[C@@H]1SC2=CC=CC=C2N(CCN(C)C)C(...,COc1ccc([C@@H]2Sc3ccccc3N(CCN(C)C)C(=O)[C@@H]2...,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,...,Inactive,Inactive,Inactive,NaN,NaN,Active,NaN,NaN,Inactive,Inactive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6867,trans-beta-Farnesene,18794-84-8,CC(C)=CCC\C(C)=C\CCC(=C)C=C,C=CC(=C)CC/C=C(\C)CCC=C(C)C,NaN,Inactive,NaN,NaN,NaN,Inactive,...,Inactive,Inactive,Inactive,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6868,{2-[2-(Ethenyloxy)ethoxy]ethoxy}ethene,764-99-8,C=COCCOCCOC=C,C=COCCOCCOC=C,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,...,Inactive,Inactive,Inactive,NaN,NaN,Inactive,NaN,NaN,Inactive,Inactive
6869,{4-[(Ethenyloxy)methyl]cyclohexyl}methanol,114651-37-5,OCC1CCC(COC=C)CC1,C=COCC1CCC(CO)CC1,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,...,Inactive,Inactive,Inactive,NaN,NaN,Inactive,NaN,NaN,Inactive,Inactive
6870,{4-[({2-[3-Fluoro-4-(trifluoromethyl)phenyl]-4...,317318-84-6,CC1=C(CSC2=CC(C)=C(OCC(O)=O)C=C2)SC(=N1)C1=CC=...,Cc1cc(SCc2sc(-c3ccc(C(F)(F)F)c(F)c3)nc2C)ccc1O...,Inactive,Active,Active,Inactive,Active,Inactive,...,Inactive,Active,Active,NaN,NaN,Active,NaN,NaN,Active,Active


In [10]:
final_df = final_df.drop(columns=["SMILES_GENERATED"])
final_df = final_df.rename(columns={"SMILES_ORIGINAL": "SMILES"})
final_df

,PREFERRED_NAME,CASRN,SMILES,TOX21_AhR_LUC_Agonist_viability_HIT CALL,TOX21_AP1_BLA_Agonist_viability_HIT CALL,TOX21_ARE_BLA_agonist_viability_HIT CALL,TOX21_Aromatase_Inhibition_viability_HIT CALL,TOX21_AR_BLA_Antagonist_viability_HIT CALL,TOX21_AR_LUC_MDAKB2_Agonist_3uM_Nilutamide_viability_HIT CALL,TOX21_AR_LUC_MDAKB2_Antagonist_0.5nM_R1881_viability_HIT CALL,...,TOX21_SBE_BLA_Antagonist_viability_HIT CALL,TOX21_SHH_3T3_GLI3_Agonist_viability_HIT CALL,TOX21_SHH_3T3_GLI3_Antagonist_viability_HIT CALL,TOX21_TRB_BLA_Agonist_Followup_viability_HIT CALL,TOX21_TRB_BLA_Antagonist_Followup_viability_HIT CALL,TOX21_TR_LUC_GH3_Antagonist_viability_HIT CALL,TOX21_TR_RXR_BLA_Agonist_Followup_viability_HIT CALL,TOX21_TR_RXR_BLA_Antagonist_Followup_viability_HIT CALL,TOX21_VDR_BLA_Agonist_viability_HIT CALL,TOX21_VDR_BLA_Antagonist_viability_HIT CALL
0,"((3,4-Dihydroxyphenyl)methylene)-Propanedinitrile",118409-57-7,OC1=C(O)C=C(C=C(C#N)C#N)C=C1,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,...,Inactive,Inactive,Inactive,NaN,NaN,Inactive,NaN,NaN,Inactive,Inactive
1,(+)-2-Methylbutyl-4-methoxybenzyladine-4'-amin...,24140-30-5,CC[C@H](C)COC(=O)\C=C\C1=CC=C(C=C1)\N=C\C1=CC=...,Inactive,Active,Inactive,Active,Active,Inactive,Inactive,...,Inactive,Inactive,Active,NaN,NaN,Inactive,NaN,NaN,Active,Active
2,(+)-Cedrol,77-53-2,[H][C@@]12CC[C@@H](C)[C@]11C[C@]([H])(C2(C)C)[...,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,...,Inactive,Inactive,Inactive,NaN,NaN,Active,NaN,NaN,Inactive,Inactive
3,(+)-Cyclazocine,7313-87-3,C[C@@H]1[C@@H]2CC3=C(C=C(O)C=C3)[C@@]1(C)CCN2C...,Inactive,NaN,NaN,Inactive,Inactive,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Inactive,NaN,NaN,NaN,NaN
4,(+)-Diltiazem,42399-41-7,COC1=CC=C(C=C1)[C@@H]1SC2=CC=CC=C2N(CCN(C)C)C(...,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,...,Inactive,Inactive,Inactive,NaN,NaN,Active,NaN,NaN,Inactive,Inactive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6867,trans-beta-Farnesene,18794-84-8,CC(C)=CCC\C(C)=C\CCC(=C)C=C,NaN,Inactive,NaN,NaN,NaN,Inactive,Inactive,...,Inactive,Inactive,Inactive,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6868,{2-[2-(Ethenyloxy)ethoxy]ethoxy}ethene,764-99-8,C=COCCOCCOC=C,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,...,Inactive,Inactive,Inactive,NaN,NaN,Inactive,NaN,NaN,Inactive,Inactive
6869,{4-[(Ethenyloxy)methyl]cyclohexyl}methanol,114651-37-5,OCC1CCC(COC=C)CC1,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,...,Inactive,Inactive,Inactive,NaN,NaN,Inactive,NaN,NaN,Inactive,Inactive
6870,{4-[({2-[3-Fluoro-4-(trifluoromethyl)phenyl]-4...,317318-84-6,CC1=C(CSC2=CC(C)=C(OCC(O)=O)C=C2)SC(=N1)C1=CC=...,Inactive,Active,Active,Inactive,Active,Inactive,Inactive,...,Inactive,Active,Active,NaN,NaN,Active,NaN,NaN,Active,Active


In [11]:
final_df.to_csv("spojeni_viability.csv", index=False)